RUN600 - Monitor infrastructure (SQL)
=====================================

Description
-----------

### Use the `runner` database

In [ ]:
use [runner]

### Get the runner pass rates for the last hour

In [ ]:
WITH last_hour([name], [passed], [failed])
AS
(
    SELECT  [name],
        (SELECT count(*) FROM [metrics] r2 WHERE r1.name = r2.name AND r2.error_level = 0 AND session_start > DATEADD(hour, -1, GETDATE())) AS [passed],
        (SELECT count(*) FROM [metrics] r2 WHERE r1.name = r2.name AND r2.error_level <> 0 AND session_start > DATEADD(hour, -1, GETDATE())) AS [failed]
    FROM [metrics] r1
    WHERE session_start > DATEADD(hour, -1, GETDATE())
    GROUP BY [name]
)
SELECT [name], [passed], [failed], [passed] + [failed] as total, CONVERT(DECIMAL(5, 2), ([passed] * 1.0) / ([passed] + [failed]) * 100.0) as [percent]
FROM last_hour

### Get the runner pass rates over last 24 hours

In [ ]:
WITH last_24_hours([name], [passed], [failed])
AS
(
    SELECT  [name],
        (SELECT count(*) FROM [metrics] r2 WHERE r1.name = r2.name AND r2.error_level = 0 AND session_start > GETDATE() - 1) AS [passed],
        (SELECT count(*) FROM [metrics] r2 WHERE r1.name = r2.name AND r2.error_level <> 0 AND session_start > GETDATE() - 1) AS [failed]
    FROM [metrics] r1
    WHERE session_start > GETDATE() - 1
    GROUP BY [name]
)
SELECT [name], [passed], [failed], [passed] + [failed] as total, CONVERT(DECIMAL(5, 2), ([passed] * 1.0) / ([passed] + [failed]) * 100.0) as [percent]
FROM last_24_hours

### Freshness check (when did the last runner reading happen). Check both the data pool, and the copy in master-pool (used for verificaiton purposes)

In [ ]:
SELECT [name], DATEDIFF(s, max(session_start), GETDATE()) as seconds_since_last_reading
FROM metrics
GROUP BY [name]
ORDER BY 2 DESC

### When was the last failures and success for each runner

In [ ]:
SELECT [name], DATEDIFF(s, max(session_start), GETDATE()) as seconds_since_last_failure
FROM metrics
WHERE error_level <> 0
GROUP BY [name]
ORDER BY 2 ASC

SELECT [name], DATEDIFF(s, max(session_start), GETDATE()) as seconds_since_last_success
FROM metrics
WHERE error_level = 0
GROUP BY [name]
ORDER BY 2 ASC

### View the failed runner results (for last 24 hours)

In [ ]:
SELECT datediff(s, session_start, [end]) as durarion, datediff(s, session_start,  GETDATE()) as seconds_ago, * FROM metrics 
WHERE error_level <> 0 AND session_start > GETDATE() - 1
ORDER BY session_start DESC

In [ ]:
print('Notebook execution complete.')